#### Fake News Classifier Using LSTM - Deep Learning

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data = data.dropna()

In [4]:
# Independent Feature
X = data.drop('label', axis=1)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [5]:
# Dependent Feature
y = data['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [6]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [7]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [8]:
voc_size = 5000

In [9]:
msgs = X.copy()
msgs.reset_index(inplace=True)

In [10]:
import nltk
import re
from nltk.corpus import stopwords

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### Data Preprocessing

In [12]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [15]:
for i in range(0, len(msgs)):
  review = re.sub('[^a-zA-Z]', ' ', msgs['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [16]:
oneHotRepr = [one_hot(word, voc_size) for word in corpus]
oneHotRepr

[[2157, 3499, 1744, 4730, 3218, 6, 4639, 1043, 3874, 3772],
 [2450, 2569, 946, 4152, 880, 160, 3039],
 [2963, 2954, 4186, 1664],
 [3484, 3359, 1429, 2442, 4560, 4339],
 [3290, 880, 1245, 1405, 2887, 2808, 880, 4494, 952, 2709],
 [1369,
  3784,
  770,
  1304,
  4602,
  2489,
  3749,
  488,
  3399,
  837,
  3372,
  2777,
  1489,
  2932,
  3039],
 [4722, 596, 960, 3824, 4788, 3182, 4240, 3153, 959, 485, 4664],
 [100, 395, 393, 4899, 4891, 3090, 2489, 389, 959, 485, 4664],
 [4433, 906, 2016, 4265, 3249, 1716, 1390, 1556, 2489, 23],
 [1226, 2289, 4248, 708, 1811, 1754, 1992, 3595],
 [1040, 390, 1325, 1295, 3640, 4666, 1065, 1887, 3299, 1012, 2643],
 [2442, 4782, 3218, 1716, 2489, 4891],
 [3834, 3885, 3110, 1476, 2122, 934, 3443, 3292, 1212],
 [3273, 1001, 731, 1884, 660, 4524, 4064, 959, 485, 4664],
 [3899, 4934, 3126, 2765, 2933, 959, 485, 4664],
 [4014, 4003, 1544, 910, 1145, 4847, 1935, 46, 4924, 1401],
 [1924, 1596, 2569],
 [2662, 3639, 4042, 4975, 2489, 1799, 4496, 3039],
 [3020, 3580,

#### **Embedding Representation**

In [17]:
# As most of the Sentences are of Diff Lengths we will Use Pad Sequence of equal their Lengths

sent_length = 20
embedded_docs = pad_sequences(oneHotRepr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 1043, 3874, 3772],
       [   0,    0,    0, ...,  880,  160, 3039],
       [   0,    0,    0, ..., 2954, 4186, 1664],
       ...,
       [   0,    0,    0, ...,  959,  485, 4664],
       [   0,    0,    0, ..., 3807, 4349,  905],
       [   0,    0,    0, ..., 4586, 3077, 2993]], dtype=int32)

#### **Creating the Model**

In [18]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [20]:
X_final.shape

(18285, 20)

In [21]:
y_final.shape

(18285,)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#### **Model Training**

In [23]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 37ms/step - loss: 0.3227 - accuracy: 0.8519 - val_loss: 0.1924 - val_accuracy: 0.9180
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1375 - accuracy: 0.9458 - val_loss: 0.2040 - val_accuracy: 0.9122
Epoch 3/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0988 - accuracy: 0.9648 - val_loss: 0.2423 - val_accuracy: 0.9198
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0723 - accuracy: 0.9744 - val_loss: 0.2373 - val_accuracy: 0.9100
Epoch 5/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0575 - accuracy: 0.9811 - val_loss: 0.2708 - val_accuracy: 0.9160
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0422 - accuracy: 0.9869 - val_loss: 0.3643 - val_accuracy: 0.9155
Epoch 7/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0308 - accuracy: 0.9904 - val_loss: 0.4236 - val_accuracy: 0.9112

#### Performance Metrics

In [24]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict_classes(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[3064,  355],
       [ 188, 2428]])

In [25]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
score

0.9100248550124275